# The Carpentries
## Programmatic assessment

This Jupyter notebook explores data related to the workshops we run and the instructors who teach.

In [ ]:
## This code toggles all code cells displaying on or off, so we can see output and markdown cells only.
## All code cells display can be toggled on/off.

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline 

## Preview the data set for all workshops
Here is a preview of all workshops the Carpentries has ever run.  The `administrator_name` column contains inconsistent data - do not use.


In [ ]:
workshops = pd.read_csv("workshops.csv")
workshops

In [ ]:
# Clean up the tag names

workshops.loc[workshops['tag_name'].str.contains("SWC"), "tag"] = "SWC"
workshops.loc[workshops['tag_name'].str.contains("DC"), "tag"] = "DC"
workshops.loc[workshops['tag_name'].str.contains("LC"), "tag"] = "LC"
workshops.loc[workshops['tag_name'].str.contains("TTT"), "tag"] = "TTT"

workshops

## Part 1: Workshop data by country

### Workshops missing country data. 

In [ ]:
# Workshops missing country data

workshops[pd.isnull(workshops['country'])]

### All countries we have ever held workshops in, with a lifetime count of workshops

In [ ]:
# All countries we have ever done work in
countries = workshops['slug'].groupby([workshops['country']])
# countries is a SeriesGroupBy object
lifetime_countries = countries.count()
lifetime_countries = lifetime_countries.to_frame().reset_index()
lifetime_countries

### Filter the whole dataframe to include lifetime non-TTT workshops through 2018 Q1 only.

In [ ]:
# Limit to non TTT workshops

workshops_nonTTT = workshops[workshops['tag'] != "TTT"]


# Limit df to workshops through 2018Q1 only

# Get all workshops through 2017
workshops_through_2017 = workshops_nonTTT[workshops_nonTTT['year'] <= 2017]

# Get all workshops for 2018 Q1
workshops_2018Q1 = workshops_nonTTT[(workshops_nonTTT['year'] == 2018) & (workshops_nonTTT['month'] <= 3)]

# Combine the above two data frames
workshops_through_2018Q1 = pd.concat([workshops_through_2017, workshops_2018Q1])
workshops_through_2018Q1 # this will be the df used for the rest of the analysis.

# This will be used later
workshops_2017Q1 = workshops_nonTTT[(workshops_nonTTT['year'] == 2017) & (workshops_nonTTT['month'] <= 3)]


workshops_through_2018Q1

### All workshops by year and country through 2018 Q1

Will need to use [pycountry](https://pypi.org/project/pycountry/) to translate two character codes into country names.

In [ ]:
workshops_by_country_year = workshops_through_2018Q1.groupby(['country', 'year'])['slug'].count().unstack()
workshops_by_country_year = workshops_by_country_year.fillna(0)
workshops_by_country_year.rename(columns={2018: '2018Q1'}, inplace=True)
workshops_by_country_year
workshops_by_country_year['total'] = workshops_by_country_year.sum(axis=1)
workshops_by_country_year

### All workshops by country in 2017 (full year)

In [ ]:
workshops_by_country_year_2017 = workshops_by_country_year[[2017]]
workshops_by_country_year_2017 = workshops_by_country_year_2017[(workshops_by_country_year_2017[2017] != 0)]
workshops_by_country_year_2017.plot.bar(title="Workshops by country, 2017")
workshops_by_country_year_2017

### All workshops by country 2018 Q1

In [ ]:
workshops_by_country_year_2018Q1 = workshops_2018Q1.groupby(['country', 'year'])['slug'].count().unstack()
workshops_by_country_year_2018Q1.rename(columns={2018:'2018Q1'}, inplace=True)
workshops_by_country_year_2017.plot.bar(title="Workshops by country, 2017")
workshops_by_country_year_2018Q1.plot.bar(title="Workshops by country, 2018 Q1")
workshops_by_country_year_2018Q1





### All workshops by country 2017 Q1

In [ ]:

# 2017 Q1 workshops by country
workshops_by_country_year_2017Q1 = workshops_2017Q1.groupby(['country', 'year'])['slug'].count().unstack()
workshops_by_country_year_2017Q1.rename(columns={2017:'2017Q1'}, inplace=True)

# 2018 Q1 workshops by country
workshops_by_country_year_2018Q1 = workshops_2018Q1.groupby(['country', 'year'])['slug'].count().unstack()
workshops_by_country_year_2018Q1.rename(columns={2018:'2018Q1'}, inplace=True)

# Concatenated table, 2017 Q1 and 2018 Q1 workshops by country
workshops_by_country_year_1718Q1 = pd.concat([workshops_by_country_year_2017Q1, workshops_by_country_year_2018Q1], axis=1)

workshops_by_country_year_2017.plot.bar(title="Workshops by country, 2017")
workshops_by_country_year_2018Q1.plot.bar(title="Workshops by country, 2018 Q1")
workshops_by_country_year_2017Q1.plot.bar(title="Workshops by country, 2017 Q1")


### Side by side view of 2017 Q1 and 2018 Q1 workshops

In [ ]:
# plt.rcParams["patch.force_edgecolor"] = False

fig = plt.figure(figsize=(12, 6)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
width = 0.25

workshops_by_country_year_1718Q1['2017Q1'].plot(kind='bar', color='lightgreen', ax=ax, width=width, position=1, edgecolor = 'green',  linewidth = 2)
workshops_by_country_year_1718Q1['2018Q1'].plot(kind='bar', color='lightblue', ax=ax, width=width, position=0, edgecolor = 'blue',  linewidth = 2)

ax.legend(["2017 Q1", "2018 Q1"]);

# ax.set_xticklabels(workshops_by_country_2018_Q1_all_countries.country)
ax.grid(True)

plt.title("Count workshops by country, Q1 2017 & Q1 2018")

plt.show()

In [ ]:
# Would like to include maps for the above data
# See http://geopandas.org/mapping.html

---
---
---

## Part 2: Workshop attendance data


## Analyzing attendance data


### Workshops missing attendance (through 2018 Q1)

These are sites that did not share attendance data with Carpentries staff.

In [ ]:
null_attendance = workshops_through_2018Q1[pd.isnull(workshops_through_2018Q1['attendance'])]
# Attendance == 0 means it was not recorded, not that no one attended
zero_attendance = workshops_through_2018Q1[(workshops_through_2018Q1['attendance'] == 0)]
missing_attendance = pd.concat([null_attendance, zero_attendance])
missing_attendance

### Workshops with attendance data through Q1 2018

In [ ]:
has_attendance_through_2018Q1 = workshops_through_2018Q1[np.isfinite(workshops_through_2018Q1['attendance'])]
has_attendance_through_2018Q1 = has_attendance_through_2018Q1[has_attendance_through_2018Q1['attendance'] > 0]
has_attendance_through_2018Q1

In [ ]:
print("FOR WORKSHOPS THROUGH Q1 2018:")
print("* COUNT ALL WORKSHOPS:", len(workshops_through_2018Q1))
print("* COUNT WORKSHOPS MISSING ATTENDANCE:", len(missing_attendance))
print("* COUNT WORKSHOPS WITH ATTENDANCE: ", len(has_attendance_through_2018Q1))
print("* APPROX ", int(round(len(missing_attendance)/len(workshops_through_2018Q1)*100)), "% OF WORKSHOPS ARE MISSING ATTENDANCE")

### Sum attendance by year

The charts below exclude workshops with missing attendance.

In [ ]:
workshops_by_attendance = has_attendance_through_2018Q1.groupby(['tag', 'year'])['attendance'].sum().unstack()
workshops_by_attendance

In [ ]:
workshops_by_attendance.plot.bar(title="Total Carpentries learners by year")

### Binned attendance total through 2018 Q1

In [ ]:
bins = pd.cut(has_attendance_through_2018Q1['attendance'], [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, np.inf])
has_attendance_through_2018Q1.groupby(bins)['attendance'].agg(['count'])

### Binned attendance counts by year.

Shows how many workshops had 1-10 learners, 11-20 learners, etc. Right side is inclusive, left side is not.
How do we show this as a proportion of total, not just as absolute count?
How do we visualize this?  This will help us see if workshops are growing or shrinking in size.

In [ ]:
# See https://stackoverflow.com/questions/34317149/pandas-groupby-with-bin-counts
# >>> groups = df.groupby(['username', pd.cut(df.views, bins)])
# >>> groups.size().unstack()

bins = pd.cut(has_attendance_through_2018Q1['attendance'], [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, np.inf])
binned_attendance_by_year = has_attendance_through_2018Q1.groupby([ bins, 'year'])['attendance'].agg(['count'])

binned_attendance_by_year =  binned_attendance_by_year.unstack()
binned_attendance_by_year['total'] = binned_attendance_by_year.sum(axis=1)

binned_attendance_by_year

# How do we rename 2018 to 2018Q1?  It's not a single index value.

---
---
---

## Part 3: Workshops

## Workshops by year

This table and chart below show the total number of workshops each year through 2018 Q1. Instructor training events are not included here.

In [ ]:
workshops_by_carpentry_year = workshops_through_2018Q1.groupby(['year', 'tag'])['slug'].count().unstack()

workshops_by_carpentry_year['total'] = workshops_by_carpentry_year.sum(axis=1)

workshops_by_carpentry_year



In [ ]:
fig = plt.figure(figsize=(12, 6)) # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
width = .5
title = "Carpentries workshops count by year"

# df.plot(x="X", y=["A", "B", "C"], kind="bar")
workshops_by_carpentry_year.plot(y = ["DC", "LC", "SWC"], kind='bar', ax=ax, width=width, position=1, title=title)

ax.grid(True)

plt.show()

## Part 4: Badge data

This section is not working - need to add new query that counts badges & people, not just badges

### All badges

The table below shows all Instructor, Trainer, or Maintainer badges issued by the Carpentries. One individual may have multiple badges.

In [ ]:
all_badges = pd.read_csv("badges_by_year_agg.csv")
all_badges

### Badges by training event

The table below lists all instructor training events run by the Carpentries, with the number attended and the number getting badged, as well as this as a percentage.

In [ ]:
badges_by_training_event = pd.read_csv("badges_by_training_event.csv")

# Get all trainings through 2017
trainings_through_2017 = badges_by_training_event[badges_by_training_event['year'] <= 2017]

# Get all trainings for 2018 Q1
trainings_2018Q1 = badges_by_training_event[(badges_by_training_event['year'] == 2018) & (badges_by_training_event['month'] <= 3)]

# Combine the above two data frames
trainings_through_2018Q1 = pd.concat([trainings_through_2017, trainings_2018Q1])

trainings_through_2018Q1['pct_completion'] = trainings_through_2018Q1['count_badged'] / trainings_through_2018Q1['attendance'] * 100
trainings_through_2018Q1 = trainings_through_2018Q1.round({'pct_completion':1})
trainings_through_2018Q1


### Completion rates by training event

In [ ]:
bins = pd.cut(trainings_through_2018Q1['pct_completion'], [0, 25, 50, 75, 99, 100])
binned_badges_by_year = trainings_through_2018Q1.groupby([ bins, 'year'])['pct_completion'].agg(['count'])

binned_badges_by_year =  binned_badges_by_year.unstack()
binned_badges_by_year['total'] = binned_badges_by_year.sum(axis=1)

binned_badges_by_year

# Need to add total row

### Badges by country, 2017

This shows what country the badged person is in, not where the training took place.  The year is when the badge was awarded, not when the training took place.  Would like to plot years side by side to show growth in each country. An individual can have multiple badges.   

In [ ]:
# Source query here needs work. It is counting badged individuals, not badges issued.

badges_by_country_by_year_awarded = pd.read_csv("badges_by_country_by_year.csv")
badges_by_country_by_year_awarded = badges_by_country_by_year_awarded.groupby(['country', 'year_awarded'])['count(id)'].sum().unstack()
badges_by_country_by_year_awarded = badges_by_country_by_year_awarded.fillna(0)
badges_by_country_by_year_awarded.rename(columns={2018: '2018Q1'}, inplace=True)
badges_by_country_by_year_awarded
badges_by_country_by_year_awarded['total'] = badges_by_country_by_year_awarded.sum(axis=1)
badges_by_country_by_year_awarded

In [ ]:
badges2017 = badges_by_country_by_year_awarded[[2017]]
badges2017 = badges2017[badges2017[2017] != 0]
badges2017

### Instructor badges by country, 2018

This shows what country the badged person is in, not where the training took place.  The year is when the badge was awarded, not when the training took place.  Would like to plot years side by side to show growth in each country. 

In [ ]:
badges_by_country_by_year_awarded2018 = badges_by_country_by_year_awarded[badges_by_country_by_year_awarded['year_awarded'] == 2018]
badges_by_country_by_year_awarded2018.plot.bar(x='country', y='count(id)', title="Badges by country, 2018")
badges_by_country_by_year_awarded2018

In [ ]:
# Instructor training events
# Count number who got badged by event

# https://data.softwarecarpentry.org/queries/80/source#table
# https://data.softwarecarpentry.org/queries/83/source

# Online events - not country specific

# Would like to break this out by country as well
# Try https://data.softwarecarpentry.org/queries/61